# A2 – Bone vs Living Community Analysis

This notebook builds a reproducible workflow to compare bone assemblages with aerial census data across Ol Pejeta Conservancy (OPC).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

try:
    import skbio
    from skbio.diversity import beta_diversity
    from skbio.stats.distance import permanova, DistanceMatrix
except ImportError as exc:  # pragma: no cover
    raise ImportError("scikit-bio is required. Install with `pip install scikit-bio`." ) from exc

from scipy.stats import pearsonr, spearmanr

from IPython.display import display

## Introduction & goals

This notebook follows the A2 analysis plan to evaluate whether the bone community reflects the living community and whether bone assemblages capture changes before and after fence removal (2007).

In [ ]:
# Configuration
CONFIG = {
    "census_path": "../data/export/excel/a2_df_census_ol_pejeta_by_sector.xlsx",
    "bones_path": "../data/export/excel/a2_df_bone_census_ol_pejeta_by_sector.xlsx",
    "species_col": "Species",
    "row_total_label": "Row_Total",
    "column_total_label": "Column_Total",
    "row_total_col": "Row_Total",
    "min_occurrence_prop": 0.2,  # species must appear in at least this proportion of samples
    "random_state": 42,
    "post_fence_year": 2007,
}

SHEET_SECTOR_MAP = {
    "Eastern": "E",
    "Western": "W",
    "All": "ALL",
    "Eastern_rank": "E",
    "Western_rank": "W",
    "All_rank": "ALL",
}

np.random.seed(CONFIG["random_state"])


## Data loading & cleaning

In [ ]:
def tidy_sheet(df: pd.DataFrame, species_col: str, value_name: str, sector: str, sample_name: str) -> pd.DataFrame:
    """Convert a wide sheet into long format with species, sample, and counts."""
    df = df.rename(columns={species_col: "species"})
    # Drop total column if present
    if CONFIG["row_total_col"] in df.columns:
        df = df.drop(columns=[CONFIG["row_total_col"]])
    # Drop total rows if present
    drop_labels = {CONFIG["row_total_label"]}
    if CONFIG.get("column_total_label"):
        drop_labels.add(CONFIG["column_total_label"])
    df = df[~df["species"].str.strip().isin(drop_labels)]

    df["species"] = df["species"].str.strip()
    df = df.melt(id_vars=["species"], var_name=sample_name, value_name=value_name)
    df[sample_name] = pd.to_numeric(df[sample_name], errors="coerce")
    df[value_name] = pd.to_numeric(df[value_name], errors="coerce")
    df = df.dropna(subset=[sample_name, value_name])
    df[value_name] = df[value_name].astype(float)
    df["sector"] = sector.upper()
    return df


def load_dataset(path: str, value_name: str, sample_name: str) -> pd.DataFrame:
    """Load all sheets from an Excel file into a long DataFrame."""
    frames = []
    xls = pd.ExcelFile(path)
    for sheet, sector in SHEET_SECTOR_MAP.items():
        if sheet not in xls.sheet_names:
            continue
        sheet_df = pd.read_excel(path, sheet_name=sheet)
        frame = tidy_sheet(sheet_df, CONFIG["species_col"], value_name=value_name, sector=sector, sample_name=sample_name)
        frame["sheet"] = sheet
        frames.append(frame)
    df = pd.concat(frames, ignore_index=True)
    df["sector"] = df["sector"].str.strip().str.upper().replace({"EAST": "E", "WEST": "W", "EASTERN": "E", "WESTERN": "W"})
    return df

# Load data
census_long = load_dataset(CONFIG["census_path"], value_name="count", sample_name="year")
bones_long = load_dataset(CONFIG["bones_path"], value_name="count", sample_name="death_year_estimate")

census_long.head(), bones_long.head()


### Filter rare species

Species that occur in fewer samples than the configured proportion are removed to stabilise composition estimates.

In [ ]:
def filter_species(
    df: pd.DataFrame,
    sample_col: str,
    species_col: str = "species",
    threshold: float | None = None,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Filter species occurring in fewer than threshold proportion of samples and report drops."""
    if threshold is None:
        threshold = CONFIG["min_occurrence_prop"]

    sample_counts = df[[sample_col, species_col]].drop_duplicates()
    total_samples = sample_counts[sample_col].nunique()
    occ = sample_counts.groupby(species_col)[sample_col].nunique()
    cutoff = threshold * total_samples

    summary = (
        occ.reset_index(name="occurrences")
        .assign(total_samples=total_samples)
    )
    summary["occurrence_prop"] = summary["occurrences"] / summary["total_samples"]
    summary["dropped"] = summary["occurrences"] < cutoff

    dropped = summary[summary["dropped"]].copy()
    dropped["reason"] = dropped.apply(
        lambda row: (
            f"Observed in {row['occurrences']} of {int(row['total_samples'])} samples "
            f"({row['occurrence_prop']:.2f}); below threshold {threshold:.2f}"
        ),
        axis=1,
    )

    keep_species = summary.loc[~summary["dropped"], species_col]
    filtered = df[df[species_col].isin(keep_species)].copy()
    return filtered, dropped[[species_col, "occurrences", "total_samples", "occurrence_prop", "reason"]]


def describe_drops(label: str, dropped_df: pd.DataFrame) -> None:
    """Display a table of dropped species with reasons."""
    if dropped_df.empty:
        print(f"No species dropped from {label}; all met the occurrence threshold.")
    else:
        display(
            dropped_df.sort_values("occurrence_prop").reset_index(drop=True)
        )


census_long, census_dropped = filter_species(census_long, sample_col="year")
bones_long, bones_dropped = filter_species(bones_long, sample_col="death_year_estimate")

print(f"Census species retained: {census_long['species'].nunique()}")
print(f"Bone species retained: {bones_long['species'].nunique()}")

print("\nDropped census species (occurrence proportion):")
describe_drops("census", census_dropped)

print("\nDropped bone species (occurrence proportion):")
describe_drops("bones", bones_dropped)


## Helper functions

In [ ]:
from typing import Iterable, Tuple


def make_species_matrix(df: pd.DataFrame, sample_cols: Iterable[str], species_col: str, count_col: str, normalize: bool = True) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Return sample × species matrix of counts and relative abundances."""
    df = df.copy()
    df[species_col] = df[species_col].str.strip()
    df[count_col] = df[count_col].fillna(0)
    df["sample_id"] = df[list(sample_cols)].astype(str).agg("_".join, axis=1)
    matrix = df.pivot_table(index="sample_id", columns=species_col, values=count_col, aggfunc="sum", fill_value=0)
    if normalize:
        rel_matrix = matrix.div(matrix.sum(axis=1).replace(0, np.nan), axis=0).fillna(0)
    else:
        rel_matrix = matrix.copy()
    return matrix, rel_matrix


def to_relative_abundance(counts: pd.Series) -> pd.Series:
    """Convert counts to relative abundance."""
    total = counts.sum()
    if total == 0:
        return counts * 0
    return counts / total


def align_vectors(vec1: pd.Series, vec2: pd.Series) -> Tuple[pd.Series, pd.Series]:
    """Align two composition vectors on shared species."""
    shared = vec1.index.intersection(vec2.index)
    return vec1.loc[shared].sort_index(), vec2.loc[shared].sort_index()


def bray_curtis_between_groups(matrix: pd.DataFrame, group_labels: pd.Series) -> float:
    """Bray–Curtis distance between mean compositions of two groups."""
    if group_labels.nunique() != 2:
        raise ValueError("Exactly two groups required")
    means = matrix.groupby(group_labels).mean()
    dm = beta_diversity("braycurtis", means.values, ids=means.index)
    return dm[0, 1]


def permanova_on_matrix(matrix: pd.DataFrame, metadata: pd.DataFrame, factor_col: str, metric: str = "braycurtis", n_permutations: int = 999):
    """Run PERMANOVA on a sample × species matrix."""
    ids = matrix.index.tolist()
    data = matrix.values
    dist = beta_diversity(metric, data, ids=ids)
    dm = DistanceMatrix(dist, ids=ids)
    result = permanova(dm, metadata, column=factor_col, permutations=n_permutations)
    return result.to_series() if hasattr(result, "to_series") else result


def composition_correlation(vec1: pd.Series, vec2: pd.Series) -> Tuple[float, float]:
    """Pearson and Spearman correlations between aligned compositions."""
    v1, v2 = align_vectors(vec1, vec2)
    pear = pearsonr(v1, v2).statistic
    spear = spearmanr(v1, v2).statistic
    return pear, spear


def permutation_null(vec1: pd.Series, vec2: pd.Series, stat_func, n: int = 999, random_state: int = None) -> np.ndarray:
    """Generate null distribution by shuffling species labels of vec2."""
    rng = np.random.default_rng(random_state)
    stats = []
    for _ in range(n):
        shuffled = pd.Series(vec2.values, index=rng.permutation(vec2.index))
        stats.append(stat_func(vec1, shuffled))
    return np.array(stats)


## Q1 – Bone vs living community (OPC-wide, 2008+)

In [ ]:
# Aggregate live data 2008+
live_opc = (census_long
            .query("year >= 2008")
            .groupby("species")["count"].sum())
live_opc_rel = to_relative_abundance(live_opc)

# Aggregate bones 2008+
bone_opc = (bones_long
            .query("death_year_estimate >= 2008")
            .groupby("species")["count"].sum())
bone_opc_rel = to_relative_abundance(bone_opc)

live_aligned, bone_aligned = align_vectors(live_opc_rel, bone_opc_rel)

bc_distance = beta_diversity("braycurtis", np.vstack([live_aligned, bone_aligned]), ids=["live", "bone"])[0, 1]
pear_corr, spear_corr = composition_correlation(live_aligned, bone_aligned)

print(f"Bray–Curtis distance (OPC 2008+): {bc_distance:.3f}")
print(f"Pearson r: {pear_corr:.3f}; Spearman rho: {spear_corr:.3f}")

# Null models
bc_null = permutation_null(live_aligned, bone_aligned, lambda v1, v2: beta_diversity("braycurtis", np.vstack([v1, v2]), ids=["live", "bone"])[0,1], n=999, random_state=CONFIG["random_state"])
pear_null = permutation_null(live_aligned, bone_aligned, lambda v1, v2: pearsonr(*align_vectors(v1, v2)).statistic, n=999, random_state=CONFIG["random_state"])

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].hist(bc_null, bins=30, color="grey", alpha=0.7)
axes[0].axvline(bc_distance, color="red", linestyle="--", label="Observed")
axes[0].set_title("Null Bray–Curtis")
axes[0].set_xlabel("Distance")
axes[0].legend()

axes[1].hist(pear_null, bins=30, color="grey", alpha=0.7)
axes[1].axvline(pear_corr, color="red", linestyle="--", label="Observed")
axes[1].set_title("Null Pearson r")
axes[1].set_xlabel("Correlation")
axes[1].legend()
plt.tight_layout()
plt.show()


## Q1 – Bone vs living community (East vs West)

In [ ]:
sector_summary = []
for sector in ["E", "W"]:
    live_sector = (census_long.query("sector == @sector")
                   .groupby("species")["count"].sum())
    bone_sector = (bones_long.query("sector == @sector")
                   .groupby("species")["count"].sum())
    live_rel = to_relative_abundance(live_sector)
    bone_rel = to_relative_abundance(bone_sector)
    live_align, bone_align = align_vectors(live_rel, bone_rel)
    bc = beta_diversity("braycurtis", np.vstack([live_align, bone_align]), ids=["live", "bone"])[0,1]
    pear, spear = composition_correlation(live_align, bone_align)
    sector_summary.append({"sector": sector, "bray_curtis": bc, "pearson": pear, "spearman": spear})

summary_df = pd.DataFrame(sector_summary)
summary_df


In [ ]:
# Cross comparisons
results = []
for bone_sector in ["E", "W"]:
    bone_comp = to_relative_abundance(bones_long.query("sector == @bone_sector").groupby("species")["count"].sum())
    for live_sector in ["E", "W"]:
        live_comp = to_relative_abundance(census_long.query("sector == @live_sector").groupby("species")["count"].sum())
        live_align, bone_align = align_vectors(live_comp, bone_comp)
        bc = beta_diversity("braycurtis", np.vstack([live_align, bone_align]), ids=["live", "bone"])[0,1]
        pear, spear = composition_correlation(live_align, bone_align)
        results.append({"bone_sector": bone_sector, "live_sector": live_sector, "bray_curtis": bc, "pearson": pear, "spearman": spear})

cross_sector = pd.DataFrame(results)
cross_sector


## Q2 – Temporal change in aerial community (pre vs post 2007)

In [ ]:
post_year = CONFIG["post_fence_year"]

census_long["period"] = np.where(census_long["year"] < post_year, "pre", "post")

permanova_results = []
period_distances = []
for sector in ["E", "W"]:
    subset = census_long.query("sector == @sector")
    matrix_counts, matrix_rel = make_species_matrix(
        subset, sample_cols=["year"], species_col="species", count_col="count", normalize=True
    )
    metadata = (
        subset.drop_duplicates(subset=["year"])
        .assign(sample_id=lambda d: d["year"].astype(str))
        .set_index("sample_id")[
            ["year", "sector", "period"]
        ]
    )
    metadata = metadata.loc[matrix_rel.index]

    perm = permanova_on_matrix(matrix_rel, metadata, factor_col="period")
    perm = perm.to_series() if hasattr(perm, "to_series") else perm
    pseudo_f = perm.get("test statistic", np.nan)
    p_value = perm.get("p-value", np.nan)
    df_between = perm.get("df", np.nan)
    df_total = (perm.get("sample size") - 1) if hasattr(perm, "index") and "sample size" in perm.index else np.nan
    r2 = np.nan
    if pd.notna(df_between) and pd.notna(df_total) and df_total > 0:
        df_within = max(df_total - df_between, 0)
        denom = (pseudo_f * df_between + df_within)
        r2 = (pseudo_f * df_between) / denom if denom != 0 else np.nan

    permanova_results.append(
        {
            "sector": sector,
            "pseudo_f": pseudo_f,
            "p_value": p_value,
            "r2": r2,
        }
    )

    bc = bray_curtis_between_groups(matrix_rel, metadata["period"])
    period_distances.append({"sector": sector, "bray_curtis_pre_post": bc})

permanova_df = pd.DataFrame(permanova_results)
period_distance_df = pd.DataFrame(period_distances)

permanova_df, period_distance_df

## Q2 – Temporal change in bones and change tracking

In [ ]:
bones_long["period"] = np.where(bones_long["death_year_estimate"] < post_year, "pre", "post")

def change_vector(df: pd.DataFrame, sector: str, value_col: str, period_col: str = "period") -> pd.Series:
    """Return post-pre change in relative abundance for a sector."""
    subset = df.query("sector == @sector")
    pre = to_relative_abundance(subset.query("%s == 'pre'" % period_col).groupby("species")[value_col].sum())
    post = to_relative_abundance(subset.query("%s == 'post'" % period_col).groupby("species")[value_col].sum())
    pre, post = align_vectors(pre, post)
    return post - pre

change_results = []
delta_store = {}
for sector in ["E", "W"]:
    delta_live = change_vector(census_long, sector, "count", period_col="period")
    delta_bone = change_vector(bones_long, sector, "count", period_col="period")
    delta_live, delta_bone = align_vectors(delta_live, delta_bone)
    pear, spear = composition_correlation(delta_live, delta_bone)

    live_rel = make_species_matrix(census_long.query("sector == @sector"), ["period"], "species", "count", normalize=True)[1]
    bone_rel = make_species_matrix(bones_long.query("sector == @sector"), ["period"], "species", "count", normalize=True)[1]
    bc_live = bray_curtis_between_groups(live_rel, live_rel.index.to_series())
    bc_bone = bray_curtis_between_groups(bone_rel, bone_rel.index.to_series())

    delta_store[sector] = (delta_live, delta_bone)
    change_results.append({
        "sector": sector,
        "delta_pearson": pear,
        "delta_spearman": spear,
        "live_pre_post_bc": bc_live,
        "bone_pre_post_bc": bc_bone,
    })

change_df = pd.DataFrame(change_results)
change_df

## Visualisations

In [ ]:
def plot_composition_bar(live_series: pd.Series, bone_series: pd.Series, title: str):
    live_df = live_series.rename("live").reset_index()
    bone_df = bone_series.rename("bone").reset_index()
    merged = pd.merge(live_df, bone_df, on="species", how="outer").fillna(0)
    merged = merged.set_index("species").sort_values("live", ascending=False)
    merged.plot(kind="bar", figsize=(12, 5))
    plt.ylabel("Relative abundance")
    plt.title(title)
    plt.tight_layout()
    plt.show()

plot_composition_bar(live_opc_rel, bone_opc_rel, "OPC-wide composition (2008+)")

# Scatter live vs bone
plt.figure(figsize=(6,6))
plt.scatter(live_aligned + 1e-6, bone_aligned + 1e-6)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Live relative abundance (log)')
plt.ylabel('Bone relative abundance (log)')
plt.title('Live vs Bone composition (OPC 2008+)')
plt.plot([live_aligned.min(), live_aligned.max()], [live_aligned.min(), live_aligned.max()], 'r--')
plt.tight_layout()
plt.show()


In [ ]:
# PCoA on aerial data
aerial_matrix_counts, aerial_matrix_rel = make_species_matrix(census_long, sample_cols=["year", "sector"], species_col="species", count_col="count", normalize=True)
aerial_metadata = census_long.drop_duplicates(subset=["year", "sector"])[["year", "sector", "period"]]
aerial_metadata["sample_id"] = aerial_metadata[["year", "sector"]].astype(str).agg("_".join, axis=1)
aerial_metadata = aerial_metadata.set_index("sample_id").loc[aerial_matrix_rel.index]

dist = beta_diversity("braycurtis", aerial_matrix_rel.values, ids=aerial_matrix_rel.index)
pcoa_res = skbio.stats.ordination.pcoa(dist)
pcoa_df = pcoa_res.samples.iloc[:, :2].join(aerial_metadata)

plt.figure(figsize=(7,6))
sns.scatterplot(data=pcoa_df, x=pcoa_df.columns[0], y=pcoa_df.columns[1], hue="sector", style="period")
plt.title("PCoA of aerial samples (Bray–Curtis)")
plt.tight_layout()
plt.show()


In [ ]:
example_sector = list(delta_store.keys())[0]
example_live, example_bone = delta_store[example_sector]

plt.figure(figsize=(6,6))
plt.scatter(example_live, example_bone)
plt.axhline(0, color='grey', linestyle='--')
plt.axvline(0, color='grey', linestyle='--')
plt.xlabel('Δ live (post - pre)')
plt.ylabel('Δ bone (post - pre)')
plt.title(f'Change tracking by species (sector {example_sector})')
plt.tight_layout()
plt.show()


## Summary of results

- **OPC-wide (2008+):** Interpret Bray–Curtis and correlations to state whether bones reflect living community.
- **East vs West:** Compare sector-specific distances to conclude closer matches.
- **Aerial pre vs post 2007:** Summarise PERMANOVA and Bray–Curtis differences.
- **Bone change tracking:** Discuss whether bone shifts align with aerial shifts in direction and magnitude.